## Importação das Bibiliotecas

In [1]:
from pyspark.sql import SparkSession
import pandas as pd
from sqlalchemy import create_engine
import pyodbc

## Criando uma SparkSession

In [2]:
spark = SparkSession.builder.appName("ReadExcel").getOrCreate()


## Processo de Leitura do Excel com Dime referente ao Dados dos Participantes



In [3]:
#Ler o arquivo em Excel em Pandas, devido o spark não ter uma função nativda
arqEnem = 'E:\Estudos\SQL\Dime\DIME_ENEM_PARTICIPANTES.xlsx'

# Ler o arquivo Excel
dfDadosGeraisPD = pd.read_excel(arqEnem, sheet_name='DIME_SEXO')


In [4]:
print(dfDadosGeraisPD)

  TP_SEXO_ID TP_SEXO_DESCRICAO
0          M         Masculino
1          F         Feminino 


In [10]:
#Foi necessário configurar este Path para realizar a conversão
import os

print(os.environ.get('PYSPARK_PYTHON'))


E:\anaconda3\python.exe


In [11]:
# Se você já tem um DataFrame do Pandas, pode converter diretamente para um DataFrame do Spark usando o método createDataFrame do SparkSession
dfDadosGerais = spark.createDataFrame(dfDadosGeraisPD)
dfDadosGerais.show()




+----------+-----------------+
|TP_SEXO_ID|TP_SEXO_DESCRICAO|
+----------+-----------------+
|         M|        Masculino|
|         F|        Feminino |
+----------+-----------------+



In [15]:
dfPreencher = dfDadosGerais.fillna(-2)  # Preenche com -2(sem ref), por exemplo


# Salvar em SQL 
Salvar os dados em uma tabela SQL


## Parametrizando a conexão do Banco SQL SERVER Usando JDBC

In [16]:
server_name = "jdbc:sqlserver://NOTEBOOK-LEO:1433"
database_name = "Enem"
table_name = "DimeSexo"
username = "bi_leonardobo"
password = "0000"
url = server_name + ";" + "databaseName=" + database_name + ";"
print(url)


jdbc:sqlserver://NOTEBOOK-LEO:1433;databaseName=Enem;


In [17]:
# Nome da nova tabela que será criada
nomeTabela = table_name

In [18]:
# Configurações do banco de dados em ODBC para dropar
server_name = "NOTEBOOK-LEO\\SQLEXPRESS"
 
# Conecte-se ao banco de dados usando pyodbc
conn = pyodbc.connect(f"Driver={{SQL Server}};Server={server_name};Database={database_name};UID={username};PWD={password}")

# Crie um cursor
cursor = conn.cursor()

# Execute a operação DROP TABLE na tabela 'Participantes'
truncate_query = f"IF OBJECT_ID('{table_name}', 'U') IS NOT NULL DROP TABLE {table_name}"
cursor.execute(truncate_query)

# Commit as alterações no banco de dados
conn.commit()

In [19]:
# Escreva os novos dados em 'Participantes' usando a conexão JDBC
dfPreencher.write \
        .format("jdbc") \
        .option("url", url) \
        .option("driver","com.microsoft.sqlserver.jdbc.SQLServerDriver")\
        .option("ssl", "true")\
        .option("trustServerCertificate", "true")\
        .option("dbtable", table_name) \
        .option("user", username) \
        .option("password", password)\
        .option("SaveMode", "overWrite").save()
  


In [20]:
print(f'Tabela {nomeTabela} criada com sucesso no banco de dados.')


Tabela DimeSexo criada com sucesso no banco de dados.
